In [5]:
import librosa
import numpy as np
import pandas as pd
from librosa import display
from scipy.io import wavfile
from sklearn.svm import SVC
from scipy.signal import spectrogram
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, GlobalAvgPool1D, Dropout, BatchNormalization, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.utils import np_utils
from keras.regularizers import l2
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from scipy.signal import decimate
from sklearn.model_selection import train_test_split
import glob
import itertools
import numpy as np
import pandas as pd

In [6]:
#read the data from the csv file
heart_data = pd.read_csv('set_b.csv')
heart_data.head()

,dataset,fname,label,sublabel
0,b,set_b/Btraining_extrastole_127_1306764300147_C...,extrastole,NaN
1,b,set_b/Btraining_extrastole_128_1306344005749_A...,extrastole,NaN
2,b,set_b/Btraining_extrastole_130_1306347376079_D...,extrastole,NaN
3,b,set_b/Btraining_extrastole_134_1306428161797_C...,extrastole,NaN
4,b,set_b/Btraining_extrastole_138_1306762146980_B...,extrastole,NaN


In [36]:
#preprocessing to get the value of the data into the list and make ready for classification
artifact=list()
extrahls=list()
murmurs=list()
normal=list()
unlabelled=list()
beats_val=list()
labels_val=list()
count=0
fi_values=list()
for i in range(len(heart_data)):
    if(heart_data['label'][i]=='extrastole'):
        if(heart_data['sublabel'][i]=='noisyextrastole'):
            filename=heart_data['fname'][i]
            filename=filename[:6]+filename[33:]
        else:
            filename=heart_data['fname'][i]
            filename=filename[:6]+filename[16:26]+'_'+filename[26:]
        audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        beats_val.append(mfccsscaled)
        labels_val.append('extrastole')
    elif(heart_data['label'][i]=='murmur'):
        if(heart_data['sublabel'][i]=='noisymurmur'):
            filename=heart_data['fname'][i]
            filename=filename[:6]+'murmur_'+filename[33:]
        else:
            filename=heart_data['fname'][i]
            filename=filename[:6]+filename[16:22]+'_'+filename[22:]
        audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        beats_val.append(mfccsscaled)
        labels_val.append('murmur')
    elif(heart_data['label'][i]=='normal'):
        if(heart_data['sublabel'][i]=='noisynormal'):
            filename=heart_data['fname'][i]
            filename=filename[:6]+'normal_'+filename[33:]
        else:
            filename=heart_data['fname'][i]
            filename=filename[:6]+filename[16:22]+'_'+filename[22:]
        audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        beats_val.append(mfccsscaled)
        labels_val.append('normal')
    else:
        count+=1
        #name='Aunlabelledtest'
        #tmp=heart_data['fname'][i]
        #val=tmp[:6]+name+tmp[6:]
        #audio, sample_rate = librosa.load(heart_data['fname'][i], res_type='kaiser_fast') 
        fi_values.append(heart_data['fname'][i])
        #mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        #mfccsscaled = np.mean(mfccs.T,axis=0)
        #beats_val.append(mfccsscaled)
        #labels_val.append('Unassigned')
        #beats_val.append(data)
        #labels_val.append('NaN')
        


In [37]:
print("Files",end=" ")
print(len(fi_values))

Files 195


In [38]:
#Convert features and corresponding classification labels into numpy arrays
X=np.array(beats_val)
y=np.array(labels_val)

In [39]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

In [40]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2)

In [41]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 4

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))


In [42]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [43]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_8 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 771       
__________

In [44]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 300
num_batch_size = 32

start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 368 samples, validate on 93 samples
Epoch 1/300
368/368 [==============================] - 1s 3ms/step - loss: 6.9365 - acc: 0.5245 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 2/300
368/368 [==============================] - 0s 239us/step - loss: 5.1245 - acc: 0.6821 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 3/300
368/368 [==============================] - 0s 242us/step - loss: 5.0883 - acc: 0.6821 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 4/300
368/368 [==============================] - 0s 233us/step - loss: 5.0369 - acc: 0.6875 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 5/300
368/368 [==============================] - 0s 225us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 6/300
368/368 [==============================] - 0s 234us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 7/300
368/368 [==============================] - 0s 238us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 8/300


Epoch 61/300
368/368 [==============================] - 0s 213us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 62/300
368/368 [==============================] - 0s 214us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 63/300
368/368 [==============================] - 0s 213us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 64/300
368/368 [==============================] - 0s 213us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 65/300
368/368 [==============================] - 0s 213us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 66/300
368/368 [==============================] - 0s 217us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 67/300
368/368 [==============================] - 0s 212us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 68/300
368/368 [==========================

368/368 [==============================] - 0s 224us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 122/300
368/368 [==============================] - 0s 215us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 123/300
368/368 [==============================] - 0s 215us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 124/300
368/368 [==============================] - 0s 216us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 125/300
368/368 [==============================] - 0s 214us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 126/300
368/368 [==============================] - 0s 214us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 127/300
368/368 [==============================] - 0s 212us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 128/300
368/368 [==============================] 

Epoch 181/300
368/368 [==============================] - 0s 214us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 182/300
368/368 [==============================] - 0s 211us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 183/300
368/368 [==============================] - 0s 215us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 184/300
368/368 [==============================] - 0s 213us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 185/300
368/368 [==============================] - 0s 213us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 186/300
368/368 [==============================] - 0s 217us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 187/300
368/368 [==============================] - 0s 216us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 188/300
368/368 [==================

Epoch 241/300
368/368 [==============================] - 0s 216us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 242/300
368/368 [==============================] - 0s 213us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 243/300
368/368 [==============================] - 0s 216us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 244/300
368/368 [==============================] - 0s 212us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 245/300
368/368 [==============================] - 0s 215us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 246/300
368/368 [==============================] - 0s 219us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 247/300
368/368 [==============================] - 0s 210us/step - loss: 5.0807 - acc: 0.6848 - val_loss: 4.3328 - val_acc: 0.7312
Epoch 248/300
368/368 [==================

Training completed in time:  0:00:25.975651


In [45]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.6847826086956522
Testing Accuracy:  0.7311827880080028


In [46]:
filename = 'set_b/murmur_noisymurmur_156_1306936373241_B1.wav' 
audio_data, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
mfccsscaled = np.mean(mfccs.T,axis=0)
predicted_vector = model.predict_classes(np.array([mfccsscaled]))
predicted_class = le.inverse_transform(predicted_vector) 
print("The predicted class is:", predicted_class[0], '\n') 
#use the filenames in fi_values to predict the class label for the unlabelled audio


The predicted class is: normal 



/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
